In [1]:
import pandas as pd

File: cluster_annotation_term.csv has basic taxonomy structure and could be coverted to something like CAS with a bit of python scripting.  



|label|name|cluster_annotation_term_set_label|parent_term_label|parent_term_set_label|term_set_order|term_order|cluster_annotation_term_set_name|color_hex_triplet|
|---|---|---|---|---|---|---|---|---|
|CS20230722_NEUR_Glut|Glut|CCN20230722_NEUR|   ||0|0|neurotransmitter|#2B93DF|
|CS20230722_CLAS_01|01 IT-ET Glut|CCN20230722_CLAS|   ||1|0|class|#FA0087|
|CS20230722_SUBC_001|001 CLA-EPd-CTX Car3 Glut|CCN20230722_SUBC|CS20230722_CLAS_01|CCN20230722_CLAS|2|0|subclass|#64c2fc|


cluster annotation term set has keys with mappings to IDs


NOT sure where division comes in 
```json
{ 
	"name": { "CAS_key": "", "comment": ""},
	"cluster_annotation_term_set_label": { "CAS_key": "", "comment": "ABC value is actually an ID for the annotation_key" },
	"parent_term_label": { "CAS_key": "parent_cell_set", "comment": "Actually the semantically loadded ID for the label"},
	"term_set_order": { "CAS_key": "rank", "comment": "Kinda..."},
	}
```
I think the mapping is rough enough that it is probably not possible to use the above JSON in code.

__Rough function spec__:

* Make dict of name to label for looking up IDs.  For parent cell set.
* Make dict lookup label: name for cluster_annotation_term_set.csv 
* Use these to roll a very basic CAS
* Pull over some ontology term mappings from WMBO (wherever a term at some level matches)

In [4]:
# Start by loading all into dataframes
cluster = pd.read_csv('../data/WMB_taxonomy_20230830/cluster.csv', sep=',')
cluster_annotation_term = pd.read_csv('../data/WMB_taxonomy_20230830/cluster_annotation_term.csv', sep=',')
cluster_annotation_term_set = pd.read_csv('../data/WMB_taxonomy_20230830/cluster_annotation_term_set.csv', sep=',')
cluster_to_cluster_annotation_membership = pd.read_csv('../data/WMB_taxonomy_20230830/cluster_to_cluster_annotation_membership.csv', sep=',')

In [14]:
cluster_annotation_term_dict = cluster_annotation_term.set_index('label').to_dict(orient='index')

In [15]:
cluster_annotation_term_dict

{'CS20230722_NEUR_Glut': {'name': 'Glut',
  'cluster_annotation_term_set_label': 'CCN20230722_NEUR',
  'parent_term_label': nan,
  'parent_term_set_label': nan,
  'term_set_order': 0,
  'term_order': 0,
  'cluster_annotation_term_set_name': 'neurotransmitter',
  'color_hex_triplet': '#2B93DF'},
 'CS20230722_NEUR_NA': {'name': nan,
  'cluster_annotation_term_set_label': 'CCN20230722_NEUR',
  'parent_term_label': nan,
  'parent_term_set_label': nan,
  'term_set_order': 0,
  'term_order': 1,
  'cluster_annotation_term_set_name': 'neurotransmitter',
  'color_hex_triplet': '#666666'},
 'CS20230722_NEUR_GABA': {'name': 'GABA',
  'cluster_annotation_term_set_label': 'CCN20230722_NEUR',
  'parent_term_label': nan,
  'parent_term_set_label': nan,
  'term_set_order': 0,
  'term_order': 2,
  'cluster_annotation_term_set_name': 'neurotransmitter',
  'color_hex_triplet': '#FF3358'},
 'CS20230722_NEUR_Dopa': {'name': 'Dopa',
  'cluster_annotation_term_set_label': 'CCN20230722_NEUR',
  'parent_term_l

In [16]:
cluster_annotation_term_set_dict = cluster_annotation_term_set.set_index('label').to_dict(orient='index')
cluster_annotation_term_set_dict

{'CCN20230722_NEUR': {'name': 'neurotransmitter',
  'description': 'Clusters are assigned based on the average expression of both neurotransmitter transporter genes and key neurotransmitter synthesizing enzyme genes.',
  'order': 0},
 'CCN20230722_CLAS': {'name': 'class',
  'description': 'The top level of cell type definition in the mouse whole brain taxonomy. It is primarily determined by broad brain region and neurotransmitter type. All cells within a subclass belong to the same class. Class provides a broader categorization of cell types.',
  'order': 1},
 'CCN20230722_SUBC': {'name': 'subclass',
  'description': 'The coarse level of cell type definition in the mouse whole brain taxonomy. All cells within a supertype belong to the same subclass. Subclass groups together related supertypes.',
  'order': 2},
 'CCN20230722_SUPT': {'name': 'supertype',
  'description': 'The second finest level of cell type definition in the mouse whole brain taxonomy. All cells within a cluster belong 

In [34]:
annotation_keys = []
orders = list(cluster_annotation_term_set['order'])
orders.sort()
max_order = orders[-1]
for k,v in cluster_annotation_term_set_dict.items():
    key = {}
    key['annotation_key'] = v['name']
    key['description'] = v['description']
    if not v['name'] == 'neurotransmitter': key['rank'] = - (v['order'] - max_order)
    annotation_keys.append(key)
annotation_keys

[{'annotation_key': 'neurotransmitter',
  'description': 'Clusters are assigned based on the average expression of both neurotransmitter transporter genes and key neurotransmitter synthesizing enzyme genes.'},
 {'annotation_key': 'class',
  'description': 'The top level of cell type definition in the mouse whole brain taxonomy. It is primarily determined by broad brain region and neurotransmitter type. All cells within a subclass belong to the same class. Class provides a broader categorization of cell types.',
  'rank': 3},
 {'annotation_key': 'subclass',
  'description': 'The coarse level of cell type definition in the mouse whole brain taxonomy. All cells within a supertype belong to the same subclass. Subclass groups together related supertypes.',
  'rank': 2},
 {'annotation_key': 'supertype',
  'description': 'The second finest level of cell type definition in the mouse whole brain taxonomy. All cells within a cluster belong to the same supertype. Supertype groups together similar

In [125]:
cluster_annotation_term_dict = cluster_annotation_term.dropna().set_index('label').to_dict(orient='index')

In [46]:
import yaml
with open('../data/BDSO_taxonomy_templates/taxonomy_details.yaml', 'r') as taxd_file:
    taxd=yaml.safe_load(taxd_file.read())
    

In [47]:
pd.read_csv('../data/WMB_taxonomy_20230830/ 

[{'Taxonomy_id': 'CS202212150',
  'Description': 'Mouse full brain BICCN taxonomy using multiple RNA-Seq datasets',
  'Reference': 'TBD',
  'PMID': ['PMID:34616066'],
  'Species': ['NCBITaxon:10090'],
  'Species_abbv': ['Mouse'],
  'Gene_abbv': ['Mmus'],
  'Brain_region': ['MBA:997'],
  'Brain_region_abbv': ['MOp'],
  'Reference_gene_list': ['Ensmusg'],
  'Root_nodes': [{'Node': 'CS202212150_6347',
    'Cell_type': 'CL:0000679',
    'Location_relation': 'has_soma_location'},
   {'Node': 'CS202212150_6348',
    'Cell_type': 'CL:0000679',
    'Location_relation': 'has_soma_location'},
   {'Node': 'CS202212150_6349',
    'Cell_type': 'CL:0000679',
    'Location_relation': 'has_soma_location'},
   {'Node': 'CS202212150_6350',
    'Cell_type': 'CL:4023064',
    'Location_relation': 'has_soma_location'},
   {'Node': 'CS202212150_6351',
    'Cell_type': 'CL:4023063',
    'Location_relation': 'has_soma_location'},
   {'Node': 'CS202212150_6352',
    'Cell_type': 'CL:0000617',
    'Location_rel

In [52]:
CS202212150_nta = pd.read_csv('../data/BDSO_taxonomy_templates/nomenclature_table_CS202212150_annotated.csv', sep=',')

In [89]:
CS202212150_nta

,cell_set_accession,original_label,cell_set_label,cell_set_preferred_alias,cell_set_aligned_alias,cell_set_additional_aliases,cell_set_structure,cell_set_ontology_tag,cell_set_alias_assignee,cell_set_alias_citation,taxonomy_id,child_cell_set_accessions,NT,MBA,projection,layer,CL
0,CS202212150_1,NaN,WMB 00001,0001 Car3 Glut_1,NaN,NaN,Brain,UBERON:0000955,Zizhen Yao,NaN,CCN202212150,NaN,GO:0061535,NaN,NaN,NaN,NaN
1,CS202212150_10,NaN,WMB 00010,0010 IT EP-CLA Glut_1,NaN,NaN,Brain,UBERON:0000955,Zizhen Yao,NaN,CCN202212150,NaN,GO:0061535,http://purl.obolibrary.org/obo/MBA_942 | http:...,PATO:0070034,NaN,NaN
2,CS202212150_100,NaN,WMB 00100,0100 L4/5 IT CTX Glut_6,NaN,NaN,Brain,UBERON:0000955,Zizhen Yao,NaN,CCN202212150,NaN,GO:0061535,http://purl.obolibrary.org/obo/MBA_688,PATO:0070034,UBERON:8440001,NaN
3,CS202212150_1000,NaN,WMB 01000,1000 LSX Prdm12 ve Gaba_2,NaN,NaN,Brain,UBERON:0000955,Zizhen Yao,NaN,CCN202212150,NaN,GO:0061534,http://purl.obolibrary.org/obo/MBA_275,NaN,NaN,NaN
4,CS202212150_1001,NaN,WMB 01001,1001 LSX Prdm12 ve Gaba_3,NaN,NaN,Brain,UBERON:0000955,Zizhen Yao,NaN,CCN202212150,NaN,GO:0061534,http://purl.obolibrary.org/obo/MBA_275,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6373,CS202212150_995,NaN,WMB 00995,0995 LSX Prdm12 ve Gaba_2,NaN,NaN,Brain,UBERON:0000955,Zizhen Yao,NaN,CCN202212150,NaN,GO:0061534,http://purl.obolibrary.org/obo/MBA_275,NaN,NaN,NaN
6374,CS202212150_996,NaN,WMB 00996,0996 LSX Prdm12 ve Gaba_2,NaN,NaN,Brain,UBERON:0000955,Zizhen Yao,NaN,CCN202212150,NaN,GO:0061534,http://purl.obolibrary.org/obo/MBA_275,NaN,NaN,NaN
6375,CS202212150_997,NaN,WMB 00997,0997 LSX Prdm12 ve Gaba_2,NaN,NaN,Brain,UBERON:0000955,Zizhen Yao,NaN,CCN202212150,NaN,GO:0061534,http://purl.obolibrary.org/obo/MBA_275,NaN,NaN,NaN
6376,CS202212150_998,NaN,WMB 00998,0998 LSX Prdm12 ve Gaba_2,NaN,NaN,Brain,UBERON:0000955,Zizhen Yao,NaN,CCN202212150,NaN,GO:0061534,http://purl.obolibrary.org/obo/MBA_275,NaN,NaN,NaN


In [50]:
cluster_annotation_term_dict

{'CS20230722_NEUR_Glut': {'name': 'Glut',
  'cluster_annotation_term_set_label': 'CCN20230722_NEUR',
  'parent_term_label': nan,
  'parent_term_set_label': nan,
  'term_set_order': 0,
  'term_order': 0,
  'cluster_annotation_term_set_name': 'neurotransmitter',
  'color_hex_triplet': '#2B93DF'},
 'CS20230722_NEUR_NA': {'name': nan,
  'cluster_annotation_term_set_label': 'CCN20230722_NEUR',
  'parent_term_label': nan,
  'parent_term_set_label': nan,
  'term_set_order': 0,
  'term_order': 1,
  'cluster_annotation_term_set_name': 'neurotransmitter',
  'color_hex_triplet': '#666666'},
 'CS20230722_NEUR_GABA': {'name': 'GABA',
  'cluster_annotation_term_set_label': 'CCN20230722_NEUR',
  'parent_term_label': nan,
  'parent_term_set_label': nan,
  'term_set_order': 0,
  'term_order': 2,
  'cluster_annotation_term_set_name': 'neurotransmitter',
  'color_hex_triplet': '#FF3358'},
 'CS20230722_NEUR_Dopa': {'name': 'Dopa',
  'cluster_annotation_term_set_label': 'CCN20230722_NEUR',
  'parent_term_l

In [57]:
CS202212150_nta_dict = CS202212150_nta.set_index('cell_set_preferred_alias').to_dict(orient='index')

Sources:

cluster_annotation_term_dict
```json
{'CS20230722_NEUR_Glut': {'name': 'Glut',
  'cluster_annotation_term_set_label': 'CCN20230722_NEUR',
  'parent_term_label': nan,
  'parent_term_set_label': nan,
  'term_set_order': 0,
  'term_order': 0,
  'cluster_annotation_term_set_name': 'neurotransmitter',
  'color_hex_triplet': '#2B93DF'},
  ```
taxd
```json
[{'Taxonomy_id': 'CS202212150',
  'Description': 'Mouse full brain BICCN taxonomy using multiple RNA-Seq datasets',
  ...
  'Root_nodes': [{'Node': 'CS202212150_6347',
    'Cell_type': 'CL:0000679',
    'Location_relation': 'has_soma_location'},
   {'Node': 'CS202212150_6348',
    'Cell_type': 'CL:0000679',
    'Location_relation': 'has_soma_location'}
```

cluster_annotation_term_set_dict
```json
{'CCN20230722_NEUR': {'name': 'neurotransmitter',
  'description': 'Clusters are assigned based on the average expression of both neurotransmitter transporter genes and key neurotransmitter synthesizing enzyme genes.',
  'order': 0}
  ```
CS202212150_nta_dict

{'0001 Car3 Glut_1': {'cell_set_accession': 'CS202212150_1',
  'original_label': nan,
...
  'cell_set_alias_assignee': 'Zizhen Yao',
  'cell_set_alias_citation': nan,
  'NT': 'GO:0061535',
  'MBA': nan,
  'projection': nan,
  'layer': nan,
  'CL': nan},
  ```

* Step1: cluster_annotation_term_dict => cell_set_label: cell_set_name
* Step2: needs to get name: cell_set accession (if we were doing this safely we'd make a compound key with cell_anntation_key, but not possible until we have the original files, so going with Q&D for now & fix when files in place.

find all unique labels - discard non-unique for now.  Then make lookup 






* Make dict of name to label for looking up IDs.  For parent cell set.
* Look up ID

In [73]:
# Find duplicate names in ABC atlas

from collections import Counter
names = [v['name'] for v in cluster_annotation_term_dict.values()]
duplicate_names = [k for k,v in Counter(names).items() if v>1]


In [71]:
root_node_CL_mappings = { n['Node']: n['Cell_type'] for n in taxd[0]['Root_nodes']}

In [132]:
# Iterate over ABC atlas, constructing CAS annotation objects using lookups to WMBO
import re
annotations = []

for k,v in cluster_annotation_term_dict.items():
    annotation = {}
    name = v['name']
    name_stripped = re.sub("^\d+ (.+)", r"\1", v['name'])
    if name_stripped in  CS202212150_nta_dict.keys():
        name = name_stripped
        print(name)
    if v['cluster_annotation_term_set_name'] == 'neurotransmitter':
        # Ideally mixed NTs would be split.  Do that later
        annotation['cell_annotation_key'] = v['cluster_annotation_term_set_name']
        annotation['cell_label']=name
        next
    if name in duplicate_names: 
        next
    if name in CS202212150_nta_dict.keys():
        annotation['cell_annotation_key'] =  v['cluster_annotation_term_set_name']
        annotation['cell_label'] = name
        cell_set_accession = CS202212150_nta_dict[name]['cell_set_accession']
        annotation['cell_set_accession'] = cell_set_accession
        if v['parent_term_label'] in cluster_annotation_term_dict.keys():
            parent_cell_set_name =  cluster_annotation_term_dict[v['parent_term_label']]['name']
            if  parent_cell_set_name in CS202212150_nta_dict.keys():
                parent_cell_set_accession = CS202212150_nta_dict[parent_cell_set_name]['cell_set_accession']
                annotation['parent_cell_set_accession'] = parent_cell_set_accession
                annotation['parent_cell_set_accession'] = parent_cell_set_accession
        else: 
            print("No mapping for parent term label: " + v['parent_term_label'])
        if cell_set_accession in root_node_CL_mappings.keys(): 
            annotation['cell_type_ontology_term_id'] = root_node_CL_mappings[cell_set_accession]
            # Are there additional mappings in 
            # would be good to add a CL name lookup
    if annotation:
        annotations.append(annotation)
    
    
        
annotations
    

IT EP-CLA Glut
No mapping for parent term label: CS20230722_CLAS_01
L5/6 IT TPE-ENT Glut
No mapping for parent term label: CS20230722_CLAS_01
L6 IT CTX Glut
No mapping for parent term label: CS20230722_CLAS_01
L5 IT CTX Glut
No mapping for parent term label: CS20230722_CLAS_01
L4/5 IT CTX Glut
No mapping for parent term label: CS20230722_CLAS_01
L2/3 IT CTX Glut
No mapping for parent term label: CS20230722_CLAS_01
L2/3 IT ENT Glut
No mapping for parent term label: CS20230722_CLAS_01
IT AON-TT-DP Glut
No mapping for parent term label: CS20230722_CLAS_01
L2 IT ENT-po Glut
No mapping for parent term label: CS20230722_CLAS_01
MEA Slc17a7 Glut
No mapping for parent term label: CS20230722_CLAS_01
COAp Grxcr2 Glut
No mapping for parent term label: CS20230722_CLAS_01
LA-BLA-BMA-PA Glut
No mapping for parent term label: CS20230722_CLAS_01
CA1-ProS Glut
No mapping for parent term label: CS20230722_CLAS_01
CA3 Glut
No mapping for parent term label: CS20230722_CLAS_01
L2 IT PPP-APr Glut
No mapping

[{'cell_annotation_key': 'subclass',
  'cell_label': 'IT EP-CLA Glut',
  'cell_set_accession': 'CS202212150_6124'},
 {'cell_annotation_key': 'subclass',
  'cell_label': 'L5/6 IT TPE-ENT Glut',
  'cell_set_accession': 'CS202212150_6125'},
 {'cell_annotation_key': 'subclass',
  'cell_label': 'L6 IT CTX Glut',
  'cell_set_accession': 'CS202212150_6126'},
 {'cell_annotation_key': 'subclass',
  'cell_label': 'L5 IT CTX Glut',
  'cell_set_accession': 'CS202212150_6127'},
 {'cell_annotation_key': 'subclass',
  'cell_label': 'L4/5 IT CTX Glut',
  'cell_set_accession': 'CS202212150_6128'},
 {'cell_annotation_key': 'subclass',
  'cell_label': 'L2/3 IT CTX Glut',
  'cell_set_accession': 'CS202212150_6129'},
 {'cell_annotation_key': 'subclass',
  'cell_label': 'L2/3 IT ENT Glut',
  'cell_set_accession': 'CS202212150_6130'},
 {'cell_annotation_key': 'subclass',
  'cell_label': 'IT AON-TT-DP Glut',
  'cell_set_accession': 'CS202212150_6132'},
 {'cell_annotation_key': 'subclass',
  'cell_label': 'L2 

In [128]:
#import re
#    print(v['name'])
#for v in cluster_annotation_term_dict.values():
#    print(re.sub("^\d+ (.+)", r"\1", v['name']))
## Lots of names still exist

set(CS202212150_nta_dict.keys()).intersection(set([re.sub("^\d+ (.+)", r"\1", v['name']) for v in cluster_annotation_term_dict.values()]))


{'ABC NN',
 'ACB-BST-FS D1 Gaba',
 'ACB-BST-FS D1 Gaba_1',
 'ACB-BST-FS D1 Gaba_2',
 'AD Serpinb7 Glut',
 'ADP-MPO Trp73 Glut',
 'ADP-MPO Trp73 Glut_1',
 'ADP-MPO Trp73 Glut_2',
 'ADP-MPO Trp73 Glut_3',
 'AHN Onecut3 Gaba',
 'AHN Onecut3 Gaba_1',
 'AHN Onecut3 Gaba_2',
 'AHN Onecut3 Gaba_3',
 'APN C1ql2 Glut',
 'APN C1ql4 Glut',
 'APN C1ql4 Glut_1',
 'ARH-PVp Tbx3 Gaba',
 'ARH-PVp Tbx3 Gaba_1',
 'ARH-PVp Tbx3 Gaba_2',
 'ARH-PVp Tbx3 Gaba_3',
 'ARH-PVp Tbx3 Glut',
 'ARH-PVp Tbx3 Glut_1',
 'ARH-PVp Tbx3 Glut_2',
 'ARH-PVp Tbx3 Glut_3',
 'AV Col27a1 Glut',
 'Astro-NT NN',
 'Astro-NT NN_1',
 'Astro-NT NN_2',
 'Astro-OLF NN',
 'Astro-OLF NN_1',
 'Astro-OLF NN_3',
 'Astro-TE NN',
 'Astro-TE NN_3',
 'BAM NN',
 'BST Tac2 Gaba',
 'BST Tac2 Gaba_1',
 'BST Tac2 Gaba_2',
 'CA1-ProS Glut',
 'CA1-ProS Glut_1',
 'CA1-ProS Glut_2',
 'CA1-ProS Glut_3',
 'CA1-ProS Glut_4',
 'CA1-ProS Glut_5',
 'CA1-ProS Glut_6',
 'CA2-FC-IG Glut',
 'CA3 Glut',
 'CA3 Glut_1',
 'CA3 Glut_2',
 'CA3 Glut_3',
 'CA3 Glut_4',


In [104]:
import re
re.sub?

Signature: re.sub(pattern, repl, string, count=0, flags=0)
Docstring:
Return the string obtained by replacing the leftmost
non-overlapping occurrences of the pattern in string by the
replacement repl.  repl can be either a string or a callable;
if a string, backslash escapes in it are processed.  If it is
a callable, it's passed the Match object and must return
a replacement string to be used.
File:      /usr/local/Cellar/python@3.9/3.9.18/Frameworks/Python.framework/Versions/3.9/lib/python3.9/re.py
Type:      function

In [106]:
re.sub("^\d+ (.+)", r"\1", "174 Tanycyte NN_3")

'Tanycyte NN_3'